In [ ]:
import sire as sr

# Positional Restraints
This section of the notebook will demonstrate new sire positional restraint functionality, building to a system in which all molecules outside a defined 'bubble' are fixed in place 

In [ ]:
mols = sr.load_test_files("ala.top", "ala.crd")
mols.view()

Use the new sire `restraints` functionality to create a `restraints` object, defining the restraints which are to be applied to the simulated system. In this case a simple positional restraint will be added to the alpha carbon of our alanine dipeptide

In [ ]:
restraints = sr.restraints.positional(mols, "resname ALA and atomname CA")

In [ ]:
restraints

In [ ]:
print(mols[0].atoms([8]))

`Restraint` can be further expanded to set the force constant `k` and the half-harmonic width `r0`, as well as set a restraint position

In [ ]:
restraints = sr.restraints.positional(
    mols,
    "resname ALA",
    k="100 kcal mol-1 A-2",
    r0="0.0 A",
)

In [ ]:
mols = (
    mols.minimisation(
        restraints=restraints,
    )
    .run()
    .commit()
)

In [ ]:
d = mols.dynamics(
    timestep="4fs",
    temperature="25oC",
    restraints=restraints,
)

In [ ]:
d.run("20ps", frame_frequency="0.5ps")

In [ ]:
mols = d.commit()
mols.view()

This functionality can be expanded to, for example, freeze all atoms outside a given distance of the ligand, effectively truncating the simulated region (note that this currently doesn't improve performance, a future update will add the ability to approximate the contributions of the frozen atoms).

This is a two-part process, first we restrain the atoms within the bubble itself in order to prevent them from leaving it, this is achieved with the `restraints.positional` functionality seen above:

In [ ]:
restraints_bubble = sr.restraints.positional(
    mols,
    "molecules within 7.5 of resname ALA",
    position=mols["resname ALA"].coordinates(),
    r0=sr.u("10 A"),
)

Next, we pass the `fixed` argument in to both minimisation and dynamics (alternatively `fixed` can be specified along with all other simulation options within `map`)

In [ ]:
mols = (
    mols.minimisation(
        restraints=restraints_bubble, fixed="not (molecules within 7.5 of molidx 0)"
    )
    .run()
    .commit()
)

In [ ]:
map = {
    "restraints": restraints_bubble,
    "fixed": "not (molecules within 7.5 of molidx 0)",
    "temperature": 300 * sr.units.kelvin,
}
d = mols.dynamics(map=map)

In [ ]:
d.run("20ps", frame_frequency="0.5ps")

In [ ]:
mols = d.commit()
mols.view()

At a lower level, restraints can be passed in the `map` argument of `sire.convert`. This creates an openMM context.

In [ ]:
omm = sr.convert.to(mols, "openmm", map={"restraints": restraints})
omm

# Higher order restraints
Sire currently also supports the addition of distance restraints, as well as Boresch restraints

In [ ]:
sr.restraints.distance(
    mols,
    atoms0=mols["resname ALA and atomname CA"],
    atoms1=mols["resname ALA and atomname N"],
)

Example of boresch restraints? Using tyk2?